In [1]:
from datasets.datamodule import DEEPscreenDataModule
from engine.system import DEEPScreenClassifier
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
import pandas as pd

In [2]:
data = pd.read_csv("../../.data/processed/CHEMBL5567.csv")

In [4]:
checkpoint_callback = ModelCheckpoint(dirpath="../../.experiments/chembl5567", save_top_k=2, monitor="val_mcc")
trainer = Trainer(callbacks=[checkpoint_callback],max_epochs=100)
model = DEEPScreenClassifier(fully_layer_1=256,fully_layer_2=32,drop_rate=0.5,learning_rate=0.0001,batch_size=32)
datamodule = DEEPscreenDataModule(data=data,target_id="chembl5567",batch_size=32,experiment_result_path="../../.experiments/chembl5564",data_split_mode="non_random_split",tmp_imgs=True)
trainer.fit(model,datamodule=datamodule)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-02-02 20:57:54:INFO     - Training this hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32)]
2024-02-02 20:57:55:INFO     - Using a total of 1876 datapoints
2024-02-02 20:57:55:INFO     - non_random_split datasets splited train=1200,validation=300,test=376

   | Name       | Type                   | Params
-------------------------------------------------------
0  | conv1      | Conv2d                 | 416   
1  | bn1        | BatchNorm2d            | 64    
2  | conv2      | Conv2d                 | 8.3 K 
3  | bn2        | BatchNorm2d            | 128   
4  | conv3      | Conv2d                 | 32.9 K
5  | bn3        | BatchNorm2d            | 256   
6  | conv4      | Conv2d                 | 32.8 K
7  | bn4        | BatchNorm2d            | 128   
8  | c

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/big/lab/sjinich/venv_che/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


RuntimeError: Input type (double) and bias type (float) should be the same